In [1]:
import os
%pwd

'/Users/songfxu/Documents/study/MLOps-End2End/research'

In [2]:
os.chdir("../")
%pwd

'/Users/songfxu/Documents/study/MLOps-End2End'

In [ ]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/54wxzhxdxsf/MLOps-End2End.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "54wxzhxdxsf"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "07aa6ee47dfdabe15e39609e5853ef42f3ad1595"

In [4]:
from pydantic import BaseModel
from pathlib import Path

class ModelEvaluationConfig(BaseModel):
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

class ModelEvaluationMgr:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN
        params = self.params.ElasticNet

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params= params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/54wxzhxdxsf/MLOps-End2End.mlflow"
        )
        return model_evaluation_config

In [6]:
import pandas as pd
import os
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [23]:
class ModelEvaluation():
    def __init__(self, config=ModelEvaluationConfig):
        self.config = config
    
    def eval_metric(self, actual, pred):
        rmse = root_mean_squared_error(actual, pred)
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        data = pd.read_csv(self.config.test_data_path)
        print(self.config.target_column)
        
        test_x = data.drop([self.config.target_column], axis=1)
        test_y = data[self.config.target_column]

        model = joblib.load(self.config.model_path)

        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():        
            predict = model.predict(test_x)
           
            rmse, mae, r2 = self.eval_metric(test_y, predict)
            scores = {"rmse": rmse, "mae": mae, "r2_score": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric('rmse',rmse)
            mlflow.log_metric('mae', mae)
            mlflow.log_metric("r2_score", r2)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(
                    model,"model",registered_model_name='ElasticNetModel'
                )
            else:
                mlflow.sklearn.log_model(
                    model,"model")


In [24]:
try:
    config = ModelEvaluationMgr().get_model_evaluation_config()
    ModelEvaluation(config=config).log_into_mlflow()
except Exception as e:
     raise e

[2025-11-17 16:17:57,202: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-11-17 16:17:57,203: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-17 16:17:57,205: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-17 16:17:57,206: INFO: common: Created directory at: artifacts]
[2025-11-17 16:17:57,206: INFO: common: Created directory at: artifacts/model_evaluation]
quality
[2025-11-17 16:17:58,086: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


2025/11/17 16:17:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run blushing-skink-708 at: https://dagshub.com/54wxzhxdxsf/MLOps-End2End.mlflow/#/experiments/0/runs/71ad07012c154b3a96b731352da8fdec
🧪 View experiment at: https://dagshub.com/54wxzhxdxsf/MLOps-End2End.mlflow/#/experiments/0


RestException: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}